In [1]:
def get_reshape(dataset):
    """Ein metode som tek inn eit datasett og gjer alle reshapings-tinga for x og y, u og v pluss ribbelista.

    Args:
        dataset (string): filnamnet på hdf5-fila.

    Returns:
        tuple: Ein tuple med x, y, U, V (2d-arrays) og ribs (original-lista)
    """
 
    x,y,Umx,Vmx,I,J,ribs = get_xyuvijribs(dataset)
    
    x_reshape = x.reshape((J,I))
    y_reshape = y.reshape((J,I))
    
    Umx_reshape = Umx.reshape((len(Umx),J,I))
    Vmx_reshape = Vmx.reshape((len(Vmx),J,I))
    
    return x_reshape, y_reshape, Umx_reshape, Vmx_reshape, ribs

def get_xyuvijribs(dataset):
    """Ein metode som hentar ut alle dataa frå ei hdf5-fil og returnerer dei som ndarrays.

    Args:
        dataset (string): filnamnet på hdf5-fila

    Returns:
        tuple: x,y,U,V (alle som 1D-array), I,J (ints) og ribbelista
    """
    with h5py.File(dataset, 'r') as f:
        Umx = np.asarray(f['Umx'])
        Vmx = np.asarray(f['Vmx'])
        (I,J) = f.attrs['I'], f.attrs['J']
        x = np.asarray(f['x'])
        y = np.asarray(f['y'])
        ribs = np.asarray(f['ribs'])
    
    return x,y,Umx,Vmx,I,J,ribs
        
def get_mean(dataset):
    _,_,Umx,Vmx,I,J,_ = get_xyuvijribs(dataset)
    
    u_bar = np.nanmean(Umx,0).reshape((J,I))
    v_bar = np.nanmean(Vmx,0).reshape((J,I))
    
    
    return u_bar, v_bar
    

In [3]:
x,y,u,v,ribs=get_reshape("data/rib50_Q40_1.hdf5")

In [4]:
x[0,:]

array([-91.738167, -90.26889 , -88.799614, -87.33033 , -85.861053,
       -84.391769, -82.922493, -81.453209, -79.983932, -78.514648,
       -77.045372, -75.576088, -74.106812, -72.637527, -71.168251,
       -69.698967, -68.229691, -66.760406, -65.29113 , -63.82185 ,
       -62.35257 , -60.883289, -59.414009, -57.944729, -56.475449,
       -55.006168, -53.536892, -52.067612, -50.598331, -49.129051,
       -47.659771, -46.190491, -44.72121 , -43.25193 , -41.78265 ,
       -40.31337 , -38.84409 , -37.374809, -35.905529, -34.436249,
       -32.966969, -31.49769 , -30.02841 , -28.55913 , -27.089849,
       -25.620569, -24.151291, -22.682011, -21.21273 , -19.74345 ,
       -18.27417 , -16.80489 , -15.33561 , -13.86633 , -12.39705 ,
       -10.92777 ,  -9.45849 ,  -7.98921 ,  -6.51993 ,  -5.05065 ,
        -3.58137 ,  -2.11209 ,  -0.64281 ,   0.82647 ,   2.29575 ,
         3.76503 ,   5.23431 ,   6.70359 ,   8.17287 ,   9.64215 ,
        11.11143 ,  12.58071 ,  14.04999 ,  15.51927 ,  16.988